# PMAC presentation

**MT.PO.01: increase total amount of service on the SHN and reliability of that service by 2024**

Metrics: total service hours, average service hours, and number of transit routes (n, %).

## Routes on the State Highway Network (SHN)

Transit routes along the SHN can be categorized into 3 groups:
1. **On SHN** - where at least 20% of the transit route runs the SHN (within 50 ft) 
2. **Intersects SHN** - where at least 35% of the transit route runs within 0.5 mile of the SHN.
3. **Other** - all other transit routes.

In [1]:
import altair as alt
import branca
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import HTML, Markdown

import B1_report_metrics as report_utils
from update_vars import ANALYSIS_DATE, BUS_SERVICE_GCS
from shared_utils import geography_utils, styleguide, map_utils
from shared_utils import calitp_color_palette as cp

hq_catalog = intake.open_catalog("../high_quality_transit_areas/*.yml")

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
df = gpd.read_parquet(
    f"{BUS_SERVICE_GCS}routes_categorized_with_delay_{ANALYSIS_DATE}.parquet")

In [24]:
df.route_type.value_counts()

3    1883
2      45
4      14
0       4
1       2
Name: route_type, dtype: int64

In [14]:
def route_type_names(row): 
    if row.route_type in ['0', '1', '2']:
        return "Rail"
    elif row.route_type == '3':
        return "Bus"
    elif row.route_type == '4':
        return "Ferry"
    else:
        return "Unknown"

df["route_type_name"] = df.apply(lambda x: route_type_names(x), axis=1)

In [18]:
df.route_type_name.value_counts()

Bus        1883
Unknown     859
Rail         51
Ferry        14
Name: route_type_name, dtype: int64

In [23]:
df[(df.route_type_name=="Unknown")].calitp_itp_id.value_counts()

282    118
278    105
294     83
360     73
273     69
61      44
194     39
293     36
226     35
208     28
474     27
167     25
16      22
372     20
148     15
135     13
108     13
18      13
42      13
279     12
217     12
331      7
389      7
238      5
281      4
473      4
4        4
246      4
118      2
110      2
127      2
350      2
315      1
Name: calitp_itp_id, dtype: int64

In [16]:
df[df.category=="on_shn"].route_type_name.value_counts()

Bus        474
Unknown    183
Name: route_type_name, dtype: int64

In [17]:
df[df.category=="intersects_shn"].route_type_name.value_counts()

Bus        1052
Unknown     440
Rail          9
Ferry         2
Name: route_type_name, dtype: int64

In [ ]:
# Should I subset to df[df._merge=="both"]?
# both means that it found a corresponding match in itp_id-route_id 
# since it's been aggregated up to route_id level (shape_id can mismatch more easily)
# Decide here, this is the subset of data I will use for rest of notebook
plot_df = df[df._merge=="both"]

## Statewide Stats  

* How many service hours are scheduled for a typical weekday for (1)?

In [ ]:
summary = report_utils.get_summary_table(plot_df)  

In [ ]:
all_hours = geography_utils.aggregate_by_geography(
    summary.assign(category="All"),
    group_cols = ["category"],
    sum_cols = ["unique_route", "total_service_hours"]
)

In [ ]:
STATEWIDE_HOURS = all_hours.total_service_hours.iloc[0]
FORMATTED_HOURS = f'{STATEWIDE_HOURS:,}' 

display(
    Markdown(
        f"### Q2 2022 ({ANALYSIS_DATE}): "
        f"{FORMATTED_HOURS} total service hours statewide"
    )
)

In [ ]:
def format_table(df: pd.DataFrame):
    col_order = ['category', 'unique_route', 'pct_unique_route',
                 'total_service_hours', 'pct_total_service_hours'
                ]

    rename_cols_dict = {
        "category": "Category",
        "unique_route": "# Routes",
        "pct_unique_route": "% Routes",
        "total_service_hours": "Total Service Hours",
        "pct_total_service_hours": "% Service Hours",
        
    }
    
    df2 = (df[col_order]
           .pipe(report_utils.sort_by_column, 
                 sort_key = ["On SHN", "Intersects SHN", "Other"])
           .rename(columns = rename_cols_dict)
          )

    table = (df2.style.format(
                subset = ["# Routes", "Total Service Hours"], 
                **{'formatter': '{:,g}'})
             .format(
                 subset=["% Routes", "% Service Hours"],
                 **{'formatter': '{:.1%}'}
             )
             .set_properties(**{'text-align': 'center'})
             .set_table_styles([dict(
                 selector='th', 
                 props=[('text-align', 'center')])  
                               ])
             .hide(axis="index")
            .to_html() 
    )
    
    display(HTML(table))
    
    
format_table(summary)

In [ ]:
# Where district is missing, it's not parallel routes
# So let's ignore those sections and focus on just on_shn and do breakdown
plot_df[(plot_df.District.isna())].category.value_counts()

In [ ]:
def by_district_on_shn_breakdown(df: pd.DataFrame) -> pd.DataFrame:
    by_district = geography_utils.aggregate_by_geography(
        df[df.category=="on_shn"],
        group_cols = ["District"],
        sum_cols = ["total_service_hours", "unique_route"]
    ).astype({"total_service_hours": int, "unique_route": int})

    by_district = (report_utils.add_percent(
        by_district, 
        ["total_service_hours", "unique_route"])
        .sort_values("District")
    )

    for c in ["pct_total_service_hours", "pct_unique_route"]:
        by_district[c] = by_district[c].round(3)
    
    return by_district

In [ ]:
by_district = by_district_on_shn_breakdown(plot_df)
by_district

In [ ]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Raleway');
@import url('https://fonts.googleapis.com/css?family=Nunito+Sans');
@import url('https://fonts.googleapis.com/css?family=Bitter');
</style>

In [ ]:
def base_bar(df: pd.DataFrame) -> alt.Chart:
    chart = (alt.Chart(df)
             .mark_bar()
             .encode(
                 x=alt.X("District:N", title="District")
             )
            )
    return chart


def make_bar(df: pd.DataFrame, y_col: str) -> alt.Chart:
    """
    Make bar chart that's total service hours or 
    average service hours by district.
    """
    y_title = f"{y_col.replace('_', ' ').title()}"
    
    if y_col == "total_service_hours":
        value_format = ",.0f"
        y_buffer = 1_400
    else:
        value_format = ",.1f"
        y_buffer = 10
    
    Y_MAX = df[y_col].max() + y_buffer
    
    bar = base_bar(df)
    
    bar = (bar.encode(
        y=alt.Y(f"{y_col}:Q", title=f"{y_title}", 
                scale=alt.Scale(domain=[0, Y_MAX]),
                axis=None
               ),
        color=alt.Color("District:N", 
                        scale=alt.Scale(
                            range=cp.CALITP_CATEGORY_BRIGHT_COLORS
                        ), legend=None
                )
             )
            )
    #https://stackoverflow.com/questions/54015250/altair-setting-constant-label-color-for-bar-chart
    text = (bar
            .mark_text(align="center", baseline="bottom",
                       color="black", dy=-5  
                      )
            .encode(text=alt.Text(y_col, format=value_format), 
                    # Set color here, because encoding for mark_text gets 
                    # superseded by alt.Color
                   color=alt.value("black"), 
                   tooltip=["District:N", 
                            alt.Tooltip("total_service_hours:Q", format=",",
                                        title="Total Service Hours"
                                       ),
                            alt.Tooltip(f"avg_service_hours:Q", format=",.1f",
                                        title='Avg Service Hours')] 
                   )
    )
      
    chart = (bar+text)
    
    chart = (styleguide.preset_chart_config(chart)
             .properties(title= {
                 "text": f"{y_title} by District",
                 "subtitle": "Routes on SHN"
            }).configure_axis(grid=False)
             .configure_view(strokeWidth=0)
             .interactive()
            )
    
    #chart.save(f"{IMG_PATH}pmac_{y_col}.png")
    
    display(chart)

In [ ]:
by_district = by_district.assign(
    avg_service_hours = by_district.total_service_hours.divide(
        by_district.unique_route).round(1)
)

metrics = [
    "total_service_hours", 
    "avg_service_hours"
]

for m in metrics:
    make_bar(by_district, m)

## Map of Parallel / On SHN / Other Routes

In [ ]:
def prep_data_for_viz():
    # This categorized_routes_with_geom pulls from merge_routelines_trips
    # and adds route's line geom
    categorized_routes_with_geom = add_route_geom(ANALYSIS_DATE)
    
    category_labeling = {
        "parallel": 0,
        "on_shn": 1,
        "other": 2,
    }
    
    gdf = categorized_routes_with_geom.assign(
        category2 = categorized_routes_with_geom.category.map(category_labeling),
    ) 
    
    # line must fall within CA
    ca = hq_catalog.ca_boundary.read().to_crs(f"EPSG: {gdf.crs.to_epsg()}")

    gdf = gpd.sjoin(
        gdf,
        ca,
        how = "inner",
        predicate = "within",
    ).drop(columns= ["index_right"])

    # Buffer to style the line, project to WGS84 for folium
    gdf = gdf.assign(
        geometry = gdf.geometry.simplify(tolerance=100).buffer(300)
    ).to_crs(geography_utils.WGS84)
    
    return gdf


gdf = prep_data_for_viz()

In [ ]:
MIN_VALUE = gdf.category2.min()
MAX_VALUE = gdf.category2.max()

#CAT_VALUES = sorted(gdf.category2.unique().tolist())

colorscale = branca.colormap.StepColormap(
    colors = [cp.CALITP_CATEGORY_BRIGHT_COLORS[0], 
              cp.CALITP_CATEGORY_BRIGHT_COLORS[1],
              cp.CALITP_CATEGORY_BRIGHT_COLORS[2]
             ],
    # play with these thresholds to get all 3 colors displayed
    # have to add an additional value beyond max_value so max_value displayed
    # don't put index=CAT_VALUES
    vmin = MIN_VALUE, vmax = MAX_VALUE, 
)

display(colorscale)

POPUP_DICT = {
    "itp_id": "ITP ID", 
    "category": "Category",
    "route_id": "Route ID", 
    "total_service_hours": "Daily Service Hours",
    "route_length_mi": "Route Length (mi)",
    "District": "District",
}

m = map_utils.make_folium_choropleth_map(
    gdf,
    plot_col = "category2",
    popup_dict = POPUP_DICT, tooltip_dict = POPUP_DICT,
    fig_width = 700, fig_height = 800,
    colorscale = colorscale,
    title = "",
    legend_name = None
)

m